In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
%matplotlib inline

criteo数据集，训练集4千5百万行，测试集6百万行，特征经过脱敏处理，[l1, l13]为连续型特征有13个，[C1,C26]为类别型特征有13个。

sample criteo数据集：是对criteo数据集的采样，有10万行数据

In [2]:
#定义column名称
cols = ['label',  'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9',
       'I10', 'I11', 'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6',
       'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25',
       'C26']

In [3]:
data = pd.read_table('G:\Datasets\dac_sample.tar\dac_sample.txt', header=None, names=cols)

In [4]:
data.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [5]:
data.shape

(100000, 40)

数据预处理：
* dense feature：log 变换
* sparse feature：embedding

In [6]:
#定义特征组
dense_feats = [f for f in cols if f[0] == "I"]
sparse_feats = [f for f in cols if f[0] == "C"]

In [7]:
#处理dense特征

def process_dense_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna(0.0)   #缺失值填充为0
    for f in feats:
        d[f] = d[f].apply(lambda x : np.log(x + 1) if x > -1 else -1)
        
    return d

In [8]:
data_dense = process_dense_feats(data, dense_feats)

In [9]:
data_dense

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13
0,0.693147,0.693147,1.791759,0.000000,7.232010,1.609438,2.772589,1.098612,5.204007,0.693147,1.098612,0.000000,1.098612
1,1.098612,0.000000,3.806662,0.693147,4.634729,2.197225,1.098612,1.098612,1.609438,0.693147,0.693147,0.000000,1.609438
2,1.098612,0.000000,0.693147,2.708050,6.643790,4.499810,1.609438,1.098612,5.505332,0.693147,1.386294,1.386294,3.828641
3,0.000000,6.795706,0.000000,0.000000,8.387768,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.386294,-1.000000,0.000000,0.000000,1.098612,0.000000,1.386294,0.000000,0.000000,0.693147,0.693147,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.693147,4.110874,3.637586,0.000000,0.693147,0.000000,1.609438,0.000000,3.178054,0.693147,1.386294,0.000000,0.000000
99996,0.000000,0.000000,2.564949,0.000000,12.061752,0.000000,0.000000,1.386294,2.397895,0.000000,0.000000,0.000000,0.000000
99997,2.397895,1.098612,0.693147,3.295837,6.180017,4.110874,2.397895,2.484907,4.110874,0.693147,0.693147,0.000000,4.094345
99998,0.000000,5.968708,3.784190,1.609438,12.752360,0.000000,0.000000,1.609438,1.609438,0.000000,0.000000,0.000000,1.609438


In [10]:
#处理sparse特征
from sklearn.preprocessing import LabelEncoder

def process_sparse_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna("-1")   #缺失值填充为-1
    for f in feats:
        label_encoder = LabelEncoder()    #这里使用了labelEnconder
        d[f] = label_encoder.fit_transform(d[f])
    return d

In [11]:
data_sparse = process_sparse_feats(data, sparse_feats)

In [12]:
data_sparse

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,229,250,43140,12137,23,5,6591,27,2,7231,...,9,2422,150,3,1163,0,2,9570,41,5604
1,229,471,19101,6423,23,11,4407,8,2,1804,...,0,1727,150,1,14512,0,2,3233,41,4215
2,74,22,489,19188,23,5,5941,8,2,2505,...,6,510,0,0,34644,6,2,2818,0,0
3,229,87,29109,4556,23,11,1449,8,2,10355,...,1,1170,0,0,15997,0,2,6971,0,0
4,300,348,34237,24514,23,1,5210,8,2,3530,...,1,390,0,0,5101,0,1,8663,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,229,283,40291,23670,23,11,1113,143,2,2505,...,9,560,150,3,5553,0,1,3818,12,4183
99996,229,471,19101,6423,59,10,4762,8,0,8180,...,9,1727,150,3,14512,0,1,3233,43,4215
99997,300,405,30268,7466,23,5,726,210,2,2505,...,0,1648,150,2,12138,0,1,2271,2,8780
99998,17,330,38421,1518,23,11,5637,75,0,3951,...,9,302,0,0,32886,0,1,6918,0,0


In [13]:
#处理后的全量数据
total_data = pd.concat([data_dense, data_sparse], axis=1)
total_data['label'] = data['label']

total_data.shape

(100000, 40)

模型训练：

FM模型（Factorization Machine），是实现中忽略偏置项$w_0$，将模型分为两部分：
* 一阶线性部分
* 二阶交叉部分

#### 一阶线性部分

In [14]:
#一阶线性部分，dense特征

import tensorflow as tf

dense_inputs = []
for f in dense_feats:
    _input = Input([1], name=f)
    dense_inputs.append(_input)

In [15]:
dense_inputs

[<tf.Tensor 'I1:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I2:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I3:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I4:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I5:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I6:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I7:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I8:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I9:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I10:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I11:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I12:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'I13:0' shape=(None, 1) dtype=float32>]

In [16]:
concat_dense_inputs = Concatenate(axis=1)(dense_inputs)
fst_order_dense_layer = Dense(1)(concat_dense_inputs)

In [17]:
#sparse特征

sparse_inputs = []
for f in sparse_feats:
    _input = Input([1], name=f)
    sparse_inputs.append(_input)

In [18]:
sparse_1d_embed = []
for _input in sparse_inputs:
    f = _input.name.split(':')[0]
    voc_size = data[f].nunique()
    _embed = Flatten()(Embedding(voc_size+1, 1, embeddings_regularizer=tf.keras.regularizers.l2(0.5))(_input))
    sparse_1d_embed.append(_embed)

In [19]:
fst_order_sparse_layer = Add()(sparse_1d_embed)

In [20]:
fst_order_sparse_layer

<tf.Tensor 'add/Identity:0' shape=(None, 1) dtype=float32>

#### 二阶交叉部分

In [21]:
#二阶特征

k = 8   #隐向量的维度

#只考虑spaese的二阶交叉
sparse_kd_embed = []
for feat, _input in enumerate(sparse_inputs):
    f = sparse_feats[feat]
    voc_size = data[f].nunique()
    _embed = Embedding(voc_size+1, k, embeddings_regularizer=tf.keras.regularizers.l2(0.7))(_input)
    sparse_kd_embed.append(_embed)

In [22]:
sparse_kd_embed

[<tf.Tensor 'embedding_26/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_27/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_28/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_29/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_30/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_31/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_32/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_33/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_34/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_35/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_36/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_37/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_38/Identity:0' shape=(None, 1, 8) dtype=float32>,
 <tf.Tensor 'embedding_39/Identity:0' shape=(None, 

FM 本身的二阶交叉项，时间复杂度是$O(n^2)$，但可以通过公式简化到$O(kn)$

In [23]:
# 1.将所有sparse的embedding拼接起来，得到(n, k)的矩阵，其中n为特征数，k为embedding的大小

concat_sparse_kd_embed = Concatenate(axis = 1)(sparse_kd_embed)  # ?, n, k  (? 代表的是batch_size)

In [24]:
# 2. 先求和再平方
sum_kd_embed = Lambda(lambda x : K.sum(x, axis=1))(concat_sparse_kd_embed)  # ? , k
square_sum_kd_embed = Multiply()([sum_kd_embed, sum_kd_embed])    #? , k

In [25]:
# 3. 先平方再求和
square_kd_embed = Multiply()([concat_sparse_kd_embed, concat_sparse_kd_embed])  #?, n, k
sum_square_kd_embed = Lambda(lambda x : K.sum(x, axis=1))(square_kd_embed)   #?, k

In [26]:
# 4. 相减再除以2
sub = Subtract()([square_sum_kd_embed, sum_square_kd_embed])
sub = Lambda(lambda x : x*0.5)(sub)
snd_order_sparse_layer = Lambda(lambda x : K.sum(x, axis=1, keepdims=True))(sub)  

#### 输出层

In [27]:
linear_part = Add()([fst_order_dense_layer, fst_order_sparse_layer])
interaction_part = snd_order_sparse_layer

In [28]:
logits = Add()([linear_part, interaction_part])
output_layer = Activation("sigmoid")(logits)

#### 编译模型

In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [30]:
model = Model(dense_inputs + sparse_inputs, output_layer)

In [31]:
plot_model(model, "fm_model.png", show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [32]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C1 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C2 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C3 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C4 (InputLayer)                 [(None, 1)]          0                                            
______________________________________________________________________________________________

In [33]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['binary_crossentropy', tf.keras.metrics.AUC(name='auc')])

#### 训练

In [34]:
from tensorflow.keras.callbacks import TensorBoard

import os

In [35]:
#logdir = './fm'   # 注意 windows下不能用这个
#在传给TensorBoard的参数log_dir中不能有中文、空格、连字符（如 “_”)，另外，路径分割符要用反斜杠。最后是用os.path.join()方法生成路径。特别注意：下划线"_"不能使用

logdir = os.path.join("fm")   
if not os.path.exists(logdir):
    os.mkdir(logdir)

#output_model_file = os.path.join(logdir,"fm_model.h5")

In [36]:
tbCallBack = TensorBoard(log_dir=logdir,
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
                 write_graph=True,  # 是否存储网络结构图
                 #write_grads=True,  # 是否可视化梯度直方图
                 write_images=True, # 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)

In [37]:
#划分测试集和验证集
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(total_data, test_size= 0.3, random_state=0)

In [38]:
train_data.shape

(70000, 40)

In [39]:
train_dense_x = [train_data[f].values for f in dense_feats]
train_sparse_x = [train_data[f].values for f in sparse_feats]

train_label = [train_data['label'].values]

In [40]:
val_dense_x = [val_data[f].values for f in dense_feats]
val_sparse_x = [val_data[f].values for f in sparse_feats]

val_label = [val_data['label'].values]

In [41]:
#训练模型
#修改了tbCallBack之后，再重启kernel，代码才能跑通

model.fit(train_dense_x+train_sparse_x, 
          train_label, epochs=5, batch_size=128,
          validation_data=(val_dense_x+val_sparse_x, val_label), 
          callbacks=[tbCallBack])

Train on 70000 samples, validate on 30000 samples
Epoch 1/5


F:\Application\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
F:\Application\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


70000/70000 [==============================] - 32s 453us/sample - loss: 32.2028 - binary_crossentropy: 0.8725 - auc: 0.4682 - val_loss: 0.5934 - val_binary_crossentropy: 0.5854 - val_auc: 0.5429
Epoch 2/5
70000/70000 [==============================] - 25s 358us/sample - loss: 0.5375 - binary_crossentropy: 0.5326 - auc: 0.6135 - val_loss: 0.5079 - val_binary_crossentropy: 0.5040 - val_auc: 0.6661
Epoch 3/5
70000/70000 [==============================] - 26s 365us/sample - loss: 0.5021 - binary_crossentropy: 0.4983 - auc: 0.6803 - val_loss: 0.4944 - val_binary_crossentropy: 0.4904 - val_auc: 0.6974
Epoch 4/5
70000/70000 [==============================] - 26s 372us/sample - loss: 0.4953 - binary_crossentropy: 0.4913 - auc: 0.6956 - val_loss: 0.4907 - val_binary_crossentropy: 0.4866 - val_auc: 0.7063
Epoch 5/5
70000/70000 [==============================] - 26s 372us/sample - loss: 0.4930 - binary_crossentropy: 0.4887 - auc: 0.7013 - val_loss: 0.4893 - val_binary_crossentropy: 0.4849 - val_a

The syntax of the command is incorrect.
